In [ ]:
#@title Carrega Bibliotecas

# Carrega Pacotes
import requests
from bs4 import BeautifulSoup, NavigableString, Tag
import numpy as np
import pandas as pd


In [ ]:
#@title Define Funcao de scrap de Cifra

def cifraSpider(artista):
  """Extrai cifras de Compositores do site Cifra Clube"""

  # Define Url de Musicas
  url = "https://www.cifraclub.com.br/" + artista + "/"
  
  # Conecta com URL
  reqs = requests.get(url)
  
  # Extrai informações de lista ordenada
  soup = BeautifulSoup(reqs.text, 'lxml')
  
  # Variavel auxiliar com letra e musica
  aux = [a['href'] for a in soup.find_all('a', {'class':"art_music-link"}, href=True)]

  # Somente Acode sem letra
  musicas = [x for x in aux if "letra" not in x]

  # Adiciona URLs
  enderecos = ['https://www.cifraclub.com.br' + musica for musica in musicas]

  # Repertorio Gil
  repertorio = {}

  for endereco in enderecos:
    
    # Conecta com URL
    reqs = requests.get(endereco)
    
    # Extrai informações de lista ordenada
    soup = BeautifulSoup(reqs.text, 'lxml')

    # Chave
    chave = endereco.replace(url,"").replace("/","")

    # Popula Repertorio
    repertorio[chave] = [element.text for s in soup.find_all('pre') for element in s.findChildren('b')]

  # Transforma em data frame
  df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in repertorio.items() ])).melt().dropna()

  # Altera nome de Colunas
  df.columns = ["Musicas", "Acordes"]

  # Escreve dados
  df.to_csv(artista + "_acordes" + ".csv")


In [ ]:
#@title Extrai repertorio Lista de Artista
artistas = ["adoniran-barbosa", "angra", "amado-batista", "banda-calypso", "claudia-leitte", "banda-eva", "alceu-valenca", "ana-carolina", "anitta", "arlindo-cruz", "baden-powell", "barao-vermelho", "beth-carvalho", "bezerra-da-silva", "bonde-do-tigrao", "bruno-e-marrone", "caetano-veloso", "capital-inicial", "cartola", "cazuza", "cassia-eller",  "chico-buarque", "chitaozinho-e-xororo", "clara-nunes", "cordel-do-fogo-encantado", "chiclete-com-banana", "asa-de-aguia", "olodum", "timbalada", "araketu", "terrasamba", "criolo", "daniela-mercury", "djavan", "dj-marlboro", "dennis-dj", "dorival-caymmi", "dominguinhos", "ednardo", "elis-regina", "emicida", "falamansa", "fernando-sorocaba", "gilbertp-gil", "gusttavo-lima", "gog", "gonzaguinha", "henrique-e-juliano", "ivete-sangalo", "jackson-do-pandeiro", "jorge-ben-jor", "jorge-mateus", "joao-nogueira", "lenine", "legiao-urbana", "los-hermanos", "luan-santana", "luiz-gonzaga", "joao-bosco", "marilia-mendonca", "marisa-monte", "marcelo-d2", "mc-catra", "mc-kevinho", "milton-nascimento", "mestre-ambrosio", "mv-bill", "nacao-zumbi", "nando-reis", "natiruts", "nelson-cavaquinho", "os-paralamas-do-sucesso", "paulinho-da-viola", "projota", "racionais-mcs", "ratos-de-porao", "raul-seixas", "o-rappa", "rzo", "charlie-brown-jr", "raimundos", "lulu-santos", "toquinho", "titas", "mutantes", "skank", "tom-jobim","elomar","sepultura", "sabotage", "seu-jorge", "trio-nordestino", "vinicius-de-moraes", "victor-leo", "ze-ramalho", "zeca-pagodinho", "zeze-di-camargo-e-luciano"]

# Executa a funcao para lista de artistas
for artista in artistas:
  cifraSpider(artista)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: Deprecation

In [ ]:
#@title Define Funcao de scrap letra

def letraSpider(artista):
  """Extrai letra de Compositores do site Cifra Club"""

  # Define Url de Musicas
  url = "https://www.cifraclub.com.br/" + artista + "/"
 
  # Conecta com URL
  reqs = requests.get(url)
  
  # Extrai informações de lista ordenada
  soup = BeautifulSoup(reqs.text, 'lxml')
  
  # Enderecos artista 
  titulo = [a['href'] for a in soup.find_all('a', {'class':"art_music-link"}, href=True)]

  # Nome de Musicas
  musicas = [x for x in titulo if "letra" not in x]

  # Adiciona URLs
  enderecos = ['https://www.cifraclub.com.br' + musica + "letra" for musica in musicas]

  # Repertorio 
  letras = {}

  for endereco in enderecos:

    # Conecta com URL
    reqs = requests.get(endereco)
      
    # Extrai informações de lista ordenada
    soup = BeautifulSoup(reqs.text, 'lxml')

    # Extrai frases de musicas
    aux = [str(element.nextSibling)  for s in soup.find_all("div", {"class": "letra"}) for element in s.findChildren('br')]

    # Separa em palavras
    aux2 = [frase.split() for frase in aux]

    # Chave
    chave = endereco.replace(url,"").replace("/","").replace("letra", "")

    # Popula Repertorio
    letras[chave] = [item for sublista in aux2 for item in sublista]

    # Transforma em data frame
    df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in letras.items() ])).melt().dropna()

    # Altera nome de Colunas
    df.columns = ["Musicas", "Palavras"]

    # Escreve dados
    df.to_csv(artista + "_letras" + ".csv")

In [ ]:
#@title Extrai repertorio Letras de Artistas
artistas = ["adoniran-barbosa", "angra", "amado-batista", "banda-calypso", "claudia-leitte", "banda-eva", "alceu-valenca", "ana-carolina", "anitta", "arlindo-cruz", "baden-powell", "barao-vermelho", "beth-carvalho", "bezerra-da-silva", "bonde-do-tigrao", "bruno-e-marrone", "caetano-veloso", "capital-inicial", "cartola", "cazuza", "cassia-eller",  "chico-buarque", "chitaozinho-e-xororo", "clara-nunes", "cordel-do-fogo-encantado", "chiclete-com-banana", "asa-de-aguia", "olodum", "timbalada", "araketu", "terrasamba", "criolo", "daniela-mercury", "djavan", "dj-marlboro", "dennis-dj", "dorival-caymmi", "dominguinhos", "ednardo", "elis-regina", "emicida", "falamansa", "fernando-sorocaba", "gilbertp-gil", "gusttavo-lima", "gog", "gonzaguinha", "henrique-e-juliano", "ivete-sangalo", "jackson-do-pandeiro", "jorge-ben-jor", "jorge-mateus", "joao-nogueira", "lenine", "legiao-urbana", "los-hermanos", "luan-santana", "luiz-gonzaga", "joao-bosco", "marilia-mendonca", "marisa-monte", "marcelo-d2", "mc-catra", "mc-kevinho", "milton-nascimento", "mestre-ambrosio", "mv-bill", "nacao-zumbi", "nando-reis", "natiruts", "nelson-cavaquinho", "os-paralamas-do-sucesso", "paulinho-da-viola", "projota", "racionais-mcs", "ratos-de-porao", "raul-seixas", "o-rappa", "rzo", "charlie-brown-jr", "raimundos", "lulu-santos", "toquinho", "titas", "mutantes", "skank", "tom-jobim","elomar","sepultura", "sabotage", "seu-jorge", "trio-nordestino", "vinicius-de-moraes", "victor-leo", "ze-ramalho", "zeca-pagodinho", "zeze-di-camargo-e-luciano"]

# Executa a funcao para lista de artistas
for artista in artistas:
  letraSpider(artista)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: Deprecation